In [ ]:
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import networkx as nx
import scipy.sparse
import numpy as np
import time

In [ ]:
#dataset = dgl.data.SSTDataset()
dataset = dgl.data.CiteseerGraphDataset()
#dataset = dgl.data.PubmedGraphDataset()
#dataset = dgl.data.CoraFullDataset()
#dataset = dgl.data.RedditDataset()
#dataset = dgl.data.rdf.AMDataset()

print(dataset[0])
g = dataset[0]
# print(g.nodes())
# N = g.nodes().numpy()
N = g.nodes()
print(N.shape)

hub_buffer = []
l_islands = []

TH_tmp = 10

In [6]:
def detect_hub(N, TH_tmp, hub_huffer):
    print(len(N))
    N = [i for i in N if i not in l_islands]
    hub_buffer = [i for i in N if g.out_degrees(i) >= TH_tmp]
    N = [i for i in N if i not in hub_buffer]
    # for id in N:
    #     # print(g.out_degrees(id))
    #     if id in l_islands:
    #         N.remove(id)
    #     elif g.out_degrees(id) >= TH_tmp:
    #         N.remove(id)
    #         hub_buffer.append(id)
    print(len(N))
    print(len(hub_buffer))
    return hub_buffer

In [ ]:
start = time.time()
hub_buffer = detect_hub(N, TH_tmp, hub_buffer)
# print(hub_buffer)
print("Time:{}".format(time.time()-start))

In [9]:
def task_assign(hub_buffer, task):
    for id in range(len(hub_buffer)):
        # neighbors = g.successors(hub_buffer[id]).numpy()
        neighbors = g.successors(hub_buffer[id])
        # task.append({"hub_id":hub_buffer[id], "neighbors": neighbor})  
        for n in neighbors:
            task.append((hub_buffer[id], n)) 
    return task

In [16]:
start = time.time()
task = []
task = task_assign(hub_buffer, task)
print("Time:{}".format(time.time()-start))

Time:0.047646522521972656


In [14]:
def TP_BFS(task, TH, c_max):
    v_global = []
    for i in range(len(task)):
        v_local = []
        h_local = []
        v_local.append(task[i][1])
        h_local.append(task[i][0])
        query = 0
        count = 1
        exit_flag = False

        while query != count:  # 如果存在未被访问的节点
            node = v_local[query]
            # print(node)
            # neighbors = g.successors(node).numpy()
            neighbors = g.successors(node)
            for n in neighbors:
                if g.out_degrees(n) < TH:     # 是否是hub node
                    if n in v_local:          # n 是否已经被engine p本地访问过
                        continue
                    elif n not in v_global:   # n 是否已经被其他engines访问过
                        count += 1
                        v_local.append(n)
                        v_global.append(n)
                        # 如果超过一个岛的最大节点数
                        if len(v_local) > c_max:
                            exit_flag = True
                            break 
                    else:
                        v_global = [i for i in v_global if i not in v_local]
                        exit_flag = True
                        break
                else:
                    h_local.append(n)
            query += 1 
            if exit_flag == True:
                break

        l_islands.append((v_local, h_local))


In [17]:
TH = 10

c_max = 100
start = time.time()
TP_BFS(task, TH, c_max)
print("Time:{}".format(time.time()-start))

Time:99.47373247146606


In [18]:
import time, threading

def loop():
    print("thread %s is running..." % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print("thread %s ended." % threading.current_thread().name)

print("thread %s is running..." % threading.current_thread().name)
t = threading.Thread(target = loop, name = 'LoopThread')
t.start()
t.join()
print("thread %s ended." % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.
